## Imports 

In [ ]:
import pandas as pd
import numpy as np

from sklearn.impute import KNNImputer

from scipy import stats

from sklearn.model_selection import GroupKFold
from sklearn.utils import check_random_state

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import FunctionTransformer

import matplotlib.pyplot as plt

from hmmlearn.hmm import GaussianHMM
from sklearn.cluster import KMeans

import seaborn as sns

from sklearn.mixture import GaussianMixture

from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler


<hr style="border:1px solid black"> </hr>

## Functions required to run code

#### Function to fill missing values

In [ ]:
def fill(series):
    if series.notna().all():
        return series
    else:
        non_na_values = series.dropna()
        if non_na_values.size > 0:
            fill_value = non_na_values.iloc[0]
            return np.where(series.isna(), np.full_like(series, fill_value), series)
        else:
            return series

#### Function to convert dataframe to sequences for HMM 

In [ ]:
def HMM_Sequences(df):

    selected_columns = df.columns[6:].tolist()
    
    # List of participant ids that will be iterated over --> copied from code above
    p_ids = df['p_id'].unique()

    # List to store individual selected_data arrays along with participant IDs
    selected_data_list = []

    # List to store sequence lengths
    sequence_lengths = []

    for p_id in p_ids:
        # Step 1: Create a boolean Series
        bool_loc = df['p_id'] == p_id
    
        # Step 2: Get indices where the condition is True
        row_index = bool_loc[bool_loc].index.values  
        
        # Step 3: Sort DataFrame by the numeric version of the sorting column in ascending order 
        df_sorted = df.loc[row_index, selected_columns + ['timepoint']].sort_values(by='timepoint', ascending=True)
        
        # Step 4: Extract data from selected columns
        selected_data = df_sorted[selected_columns].values.round(3)

        # Append selected_data to the list
        selected_data_list.append(selected_data)
        
        # Append the length of the selected_data to the sequence_lengths list
        sequence_lengths.append(len(selected_data))
        
    # Concatenate the sequences into a single array
    final_concatenated_data = np.concatenate(selected_data_list)

    # Array of sequence lengths
    sequence_lengths_array = np.array(sequence_lengths)
    
    return final_concatenated_data, sequence_lengths_array

#### Functioning for GMM/HMM with seed optimisation

In [ ]:
def clustering_with_seed(data_dict, n_clusters, cluster_method = 'GMM'):
    
    # Get data from dictionary
    if cluster_method == 'GMM':
        train_data = data_dict.get("train_data")
    
    elif cluster_method == 'HMM':  
        train_sequences = data_dict.get("train_sequences")
        train_sequence_lengths = data_dict.get("train_sequence_lengths")
        
    ll = []  # To store log-likelihood for each seed
    seeds = np.arange(1, 100)  # Seeds from 1 to 99 (inclusive)
    
    # Loop over each seed
    for seed in seeds:
        
        if cluster_method == 'GMM':
            # Initialize the model
            gmm = GaussianMixture(n_components=n_clusters, init_params='k-means++', covariance_type='diag', random_state=seed)
            # Fit the model
            gmm.fit(train_data)
            # Calculate log-likelihood (LL) for the current seed
            logl = gmm.score(train_data)
             
        elif cluster_method == 'HMM':  
            # Initialize the model
            model = GaussianHMM(n_components=n_clusters, covariance_type="diag", random_state = seed) ## ADDED
            # Fit the model
            model.fit(train_sequences, train_sequence_lengths)  # Train with lengths
            # Calculate log-likelihood (LL) for the current seed
            logl = model.score(train_sequences, train_sequence_lengths)
           
        # append score to list   
        ll.append(logl)
            
    # Find the median log-likelihood
    median_ll = np.median(ll)
        
    # Find the seed associated with the median LL
    median_seed = seeds[np.argmin(np.abs(ll - median_ll))]  # Seed with LL closest to the median LL
        
    # Refit the model with the best (median) seed
    if cluster_method == 'GMM':
        # re-initialize the model
        median_model = GaussianMixture(n_components=n_clusters, init_params='k-means++', covariance_type='diag', random_state=median_seed)
        # Re-fit the model
        median_model.fit(train_data)  # Fit again with the median seed
        
    elif cluster_method == 'HMM':  
        # re-initialize the model
        median_model = GaussianHMM(n_components=n_clusters, covariance_type="diag", random_state = median_seed) ## ADDED
        # Re-fit the model
        median_model.fit(train_sequences, train_sequence_lengths)  # Train with lengths
        
    return median_model

<hr style="border:1px solid black"> </hr>

## Data Preperation 

#### Read in data and organise

In [ ]:
df = pd.read_csv("radar_ids_features_hr_activity_steps.csv")
df = df.rename(columns={'record_name': 'timepoint'})

# Apply the function to the 'age' column grouped by 'p_id'
df['age_all'] = df.groupby('p_id')['age'].transform(fill)
df['age_all'] = df['age_all'] / 10

# Apply the function to the 'gender' column grouped by 'p_id'
df['gender_all'] = df.groupby('p_id')['gender'].transform(fill)

# Data selection & cleaning
total_data = df[['p_id', 'timepoint', 'act_day', 'IDS_TOTAL', 'age_all', 'gender_all',
                 'Sedentary_time_mean', 'Light_activity_mean', 'Moderate_activity_mean',
                 'Vigorous_activity_mean', 'Nighttime_activity_mean',
                 'Total_Daily_Calories_mean',
                 'Sedentary_time_sd', 'Light_activity_sd',
                 'Moderate_activity_sd', 'Vigorous_activity_sd', 'Nighttime_activity_sd',
                 'Total_Daily_Calories_sd'
                 ]]

# Drop rows with act_available >= 3
total_data = total_data[total_data['act_day'] >= 3]

#Record ID rename
replacements = {
    "12_month_assessmen_arm_1": 12,
    "15_month_assessmen_arm_1": 15,
    "18_month_assessmen_arm_1": 18,
    "21_month_assessmen_arm_1": 21,
    "24_month_assessmen_arm_1": 24,
    "27_month_assessmen_arm_1": 27,
    "30_month_assessmen_arm_1": 30,
    "33_month_assessmen_arm_1": 33,
    "36_month_assessmen_arm_1": 36,
    "39_month_assessmen_arm_1": 39,
    "3_month_assessment_arm_1": 3,
    "6_month_assessment_arm_1": 6,
    "9_month_assessment_arm_1": 9,
    "enrolment_arm_1": 0
}

# Use map function
total_data.timepoint = total_data.timepoint.map(replacements)

#### Visualise missing values 

In [ ]:
# Set the display option to show all rows
pd.set_option('display.max_rows', None)
missing_values_df = total_data.isna().sum().reset_index()
non_missing_values_df = total_data.notna().sum().reset_index(drop=True)

# Combine both into a single DataFrame
na_df = missing_values_df
na_df.columns = ['Variable', 'Missing Values']
na_df['Non-missing Values'] = non_missing_values_df
print(na_df)

# Reset the setting to default after printing
pd.reset_option('display.max_rows')

#### Impute missing values 

In [ ]:
raw_data = total_data.copy()

# Separate the participant ID and other columns without missing data
non_numeric_cols = total_data[['p_id', 'gender_all']]  
numeric_cols = total_data.drop(columns=['p_id', 'gender_all'])  

# Impute 
imputer = KNNImputer(n_neighbors=5)
imputed_numeric_data = imputer.fit_transform(numeric_cols)

# Convert the imputed numeric data back to a DataFrame
imputed_numeric_df = pd.DataFrame(imputed_numeric_data, columns=numeric_cols.columns)
imputed_df = pd.concat([non_numeric_cols.reset_index(drop=True), imputed_numeric_df], axis=1)

total_data = imputed_df.copy()

In [ ]:
# DOUBLE CHECK ALL REMOVED
pd.set_option('display.max_rows', None)
missing_values_df = total_data.isna().sum().reset_index()
non_missing_values_df = total_data.notna().sum().reset_index(drop=True)
na_df = missing_values_df
na_df.columns = ['Variable', 'Missing Values']
na_df['Non-missing Values'] = non_missing_values_df
print(na_df)
pd.reset_option('display.max_rows')

<hr style="border:1px solid black"> </hr>

## CLUSTERING

In [ ]:
# Initialize Scalers
scaler = StandardScaler()

# Range of states to evaluate
cluster_options =  [2, 3, 4, 5, 6, 7, 8, 9 ,10]

#Setting up stratified CFV
n_folds = 5
group_id = pd.factorize(total_data['p_id'])[0]
GFK = GroupKFold(n_splits=n_folds)

fold_test_scores_GMM = np.zeros([n_folds, len(cluster_options)])
fold_test_scores_HMM = np.zeros([n_folds, len(cluster_options)])

# Loop over each choice of number of states
cluster_counter = 0
for n_clusters in cluster_options:
    
    fold_counter = 0
    print('Testing number of cluster: ' + str(n_clusters))
    
    # Do repeated cross-fold validation
    for train_index, test_index in GFK.split(total_data, groups = group_id):
        
        # define data splits 
        train_data = total_data.iloc[train_index,:]
        test_data = total_data.iloc[test_index,:]
        
        # transform the data
        train_data.iloc[:,6:] = scaler.fit_transform(train_data.iloc[:,6:])
        test_data.iloc[:,6:] = scaler.transform(test_data.iloc[:,6:])
        
        # Choose GMM
        cluster_method = 'GMM'
        
        # organise training data, put in dictionary to pass to seed function 
        selected_columns = total_data.columns[6:].tolist()
        train_data_GMM = train_data[selected_columns]
        data_dict = {"train_data": train_data_GMM}
        
        # find median_fit model 
        median_model_GMM = clustering_with_seed(data_dict, n_clusters, cluster_method)
        
        # organise test data, get test set LL on  median_fit model
        test_data_GMM = test_data[selected_columns]
        fold_test_scores_GMM[fold_counter,cluster_counter] = median_model_GMM.score(test_data_GMM)
        
        # Choose HMM
        cluster_method = 'HMM'
        
        # organise training sequences, put in dictionary to pass to seed function 
        train_sequences, train_sequence_lengths = HMM_Sequences(train_data)
        data_dict = {"train_sequences": train_sequences,
                     "train_sequence_lengths": train_sequence_lengths}
        
        # find median_fit model 
        median_model_HMM = clustering_with_seed(data_dict, n_clusters, cluster_method)
        
        # organise test sequences, get test set LL on  median_fit model
        test_sequnces, test_sequence_lengths = HMM_Sequences(test_data)
        fold_test_scores_HMM[fold_counter,cluster_counter] = median_model_HMM.score(test_sequnces, test_sequence_lengths)
        
        fold_counter +=1
        
    cluster_counter += 1

In [ ]:
fig, axs = plt.subplots(2,1, sharex=True)
fig.suptitle("Activity - Average Log-Likelihood vs. Number of Cluster")

CFV_scores_GMM = np.mean(fold_test_scores_GMM,0) 
axs[0].plot(cluster_options, CFV_scores_GMM, marker = 'o', color='r', label='GMM')
axs[0].grid(True)
axs[0].set_ylabel("Average Log-Likelihood")
axs[0].set_title("GMM")

CFV_scores_HMM = np.mean(fold_test_scores_HMM,0)
axs[1].plot(cluster_options, CFV_scores_HMM, marker = 'o', color='g', label='HMM')
axs[1].grid(True)
axs[1].set_xticks(range(min(cluster_options),len(cluster_options)+min(cluster_options))) 
axs[1].set_xticklabels(cluster_options)
axs[1].set_xlabel("Number of clusters")
axs[1].set_ylabel("Average Log-Likelihood")
axs[1].set_title("HMM")

In [ ]:
print(CFV_scores_GMM)
print(CFV_scores_HMM)

In [ ]:
# Create a dictionary for the data
act_selection = {
    "Clusters": cluster_options,
    "ACT_LL_GMM": CFV_scores_GMM,
    "ACT_LL_HMM": CFV_scores_HMM
}

# Create a DataFrame from the dictionary
plotdata = pd.DataFrame(act_selection)

# Save the DataFrame to a CSV file
#plotdata.to_csv("xxx.csv", index=False)

### Final Model Run 

In [ ]:
# transform the data
total_data_scaled = total_data.copy()

scaler = StandardScaler()
total_data_scaled.iloc[:,6:18] = scaler.fit_transform(total_data.iloc[:,6:18])

In [ ]:
print(total_data_scaled.iloc[:,6:].columns)
print(total_data_scaled.iloc[:, 6:18].columns)

#### Final GMM

In [ ]:
# Choose GMM and number of clusters 
cluster_method = 'GMM'
optimal_num_clusters = 3
        
# Organise data, put in dictionary to pass to seed function 
selected_columns = total_data_scaled.iloc[:, 6:18].columns.tolist() ##### CHANGED BY ME
total_data_scaled_GMM = total_data_scaled[selected_columns]
data_dict = {"train_data": total_data_scaled_GMM}

# find median_fit model 
final_model_GMM = clustering_with_seed(data_dict, optimal_num_clusters, cluster_method)

# Predict cluster labels for the scaled dataset
final_labels = final_model_GMM.predict(total_data_scaled_GMM)

# Add the predicted labels as a new column in your dataframe
total_data_scaled['gmm'] = final_labels

# Calculate the number of unique participants
num_unique_participants = total_data_scaled['p_id'].nunique()
print("Number of unique participants:", num_unique_participants)

# Group by the predicted labels and calculate mean statistics
state_profiles = total_data_scaled.groupby('gmm').agg({
    'Sedentary_time_mean': 'mean', 
    'Light_activity_mean': 'mean',
    'Moderate_activity_mean': 'mean', 
    'Vigorous_activity_mean': 'mean',
    'Nighttime_activity_mean': 'mean',
    'Total_Daily_Calories_mean': 'mean', 
    'Sedentary_time_sd': 'mean', 
    'Light_activity_sd': 'mean', 
    'Moderate_activity_sd': 'mean',
    'Vigorous_activity_sd': 'mean', 
    'Nighttime_activity_sd': 'mean',
    'Total_Daily_Calories_sd': 'mean',
}).reset_index()

# Add a column for the number of sequences in each cluster
state_profiles['Num_Sequences'] = total_data_scaled.groupby('gmm').size().values
display(state_profiles)



In [ ]:
print(total_data_scaled.iloc[:, 0:18].columns)

#### Final HMM

In [ ]:
# Choose HMM
cluster_method = 'HMM'
optimal_num_clusters = 3

total_sequences, total_sequence_lengths = HMM_Sequences(total_data_scaled.iloc[:, 0:18])
data_dict = {"train_sequences": total_sequences,
             "train_sequence_lengths": total_sequence_lengths}

# find median_fit model 
final_model_HMM = clustering_with_seed(data_dict, optimal_num_clusters, cluster_method)

# Predict the most likely state sequence for the entire dataset
predicted_labels = final_model_HMM.predict(total_sequences, total_sequence_lengths)

# Add predicted labels to your dataframe (if you want to attach them to the original data)
total_data_scaled['hmm'] = predicted_labels

# Calculate the number of unique participants
num_unique_participants = total_data_scaled['p_id'].nunique()
print("Number of unique participants:", num_unique_participants)

# Group by the predicted labels and calculate mean statistics
state_profiles = total_data_scaled.groupby('hmm').agg({
    'Sedentary_time_mean': 'mean', 
    'Light_activity_mean': 'mean',
    'Moderate_activity_mean': 'mean', 
    'Vigorous_activity_mean': 'mean',
    'Nighttime_activity_mean': 'mean',
    'Total_Daily_Calories_mean': 'mean', 
    'Sedentary_time_sd': 'mean', 
    'Light_activity_sd': 'mean', 
    'Moderate_activity_sd': 'mean',
    'Vigorous_activity_sd': 'mean', 
    'Nighttime_activity_sd': 'mean',
    'Total_Daily_Calories_sd': 'mean',
}).reset_index()

# Add a column for the number of sequences in each cluster
state_profiles['Num_Sequences'] = total_data_scaled.groupby('hmm').size().values
display(state_profiles)

#### HMM Transition Probabilities

In [ ]:
# Access the transition matrix
transition_probabilities = final_model_HMM.transmat_
rounded_transition_probabilities = np.round(transition_probabilities, 3)
print("Rounded Transition Probabilities:")
print(rounded_transition_probabilities)

#### HMM State Probabilities

In [ ]:
# WRITE DATA FOR PREDICTION ANALYSIS 
# Get the posterior probabilities for each state at each timepoint
state_probabilities = final_model_HMM.predict_proba(total_sequences, total_sequence_lengths)

state_prob_df = pd.DataFrame(state_probabilities, columns=[f'State_Prob_{i}' for i in range(optimal_num_clusters)])
state_prob_df['timepoint'] = total_data_scaled['timepoint'].values
state_prob_df['p_id'] = total_data_scaled['p_id'].values
display(state_prob_df)

total_data_scaled_probs = pd.merge(state_prob_df, total_data_scaled, on=['p_id', 'timepoint'], how='inner')


### Relabelling/Reordering the classes for GMM & HMM for Descriptives & prediction work

In [ ]:
# RELABELING THE CLASSES: 
# Define the mapping (old value -> new value)
hmm_mapping = {0: 2, 
               1: 3, 
               2: 1} 
gmm_mapping = {0: 3, 
               1: 2, 
               2: 1} 

# Apply the mapping to the 'Category' column
total_data_scaled['hmm'] = total_data_scaled['hmm'].replace(hmm_mapping)
total_data_scaled['gmm'] = total_data_scaled['gmm'].replace(gmm_mapping)

# For the probs df
total_data_scaled_probs['hmm'] = total_data_scaled_probs['hmm'].replace(hmm_mapping)
total_data_scaled_probs['gmm'] = total_data_scaled_probs['gmm'].replace(gmm_mapping)

# Rename columns using a dictionary
total_data_scaled_probs.rename(columns={'State_Prob_0': 'State_Prob_2', 
                                        'State_Prob_1': 'State_Prob_3', 
                                        'State_Prob_2': 'State_Prob_1'}, inplace=True)

# Print the DataFrame after renaming columns
print("\nDataFrame after renaming columns:")
print(total_data_scaled_probs.columns)

In [ ]:
# Calculate frequency of values in the 'Category' column
frequency = total_data_scaled_probs['hmm'].value_counts()
print(frequency)

frequency_gmm = total_data_scaled['gmm'].value_counts()
print(frequency_gmm)

<hr style="border:1px solid black"> </hr>

### Demographic & Clinical Differences of the groups 

In [ ]:
from tableone import TableOne
import pandas as pd

total_data_scaled_copy = total_data_scaled.copy()

# Load the necessary data
chara = pd.read_csv("chara_fvfvfv.csv")
age = pd.read_csv("age.csv")
comorbidities = pd.read_csv("Comorbities_Data_only.csv")
chara = chara[['p_id', 'timepoint', 'mh_family_TOTAL', 'gender_all', 
               'mh_family_depr___0', 'mh_family_depr___1', 'mh_family_depr___2',
               'mh_longst_illness', 'LIFETIME_TRAUMA', 'WSAS_TOTAL', 'GAD7_TOTAL',
               'IDS_TOTAL', 'IDS_CAT']]


eth = pd.read_csv("Ethnicity_data.csv")
site = pd.read_csv("Site_data.csv")  # 1 = London, 2=Spain, 3=Netherlands
site['recruitmentsite'] = site['recruitmentsite'].astype('category')

emp = pd.read_csv("employment_status.csv")
emp = emp[['p_id', 'EMPLOYMENT_STATUS']]
emp['EMPLOYMENT_STATUS'] = emp['EMPLOYMENT_STATUS'].astype('category')

# Remove duplicates to ensure unique participant IDs
chara_desc_1p = chara.drop_duplicates(subset='p_id')
age_desc_1p = age.drop_duplicates(subset='p_id')

# Merge all relevant datasets
merged_allPs = pd.merge(chara_desc_1p, site, on='p_id', how='left')
merged_allPs = pd.merge(merged_allPs, eth, on='p_id', how='left')
merged_allPs = pd.merge(merged_allPs, emp, on='p_id', how='left')
merged_allPs = pd.merge(merged_allPs, comorbidities, on='p_id', how='left') 
merged_allPs = pd.merge(merged_allPs, age_desc_1p, on='p_id', how='left') 

# Select the columns you want to merge from merged_allPs
columns_to_merge = ['p_id', 'hmm']
merged_selected = total_data_scaled_copy[columns_to_merge]
total_data_withcara = pd.merge(total_data_scaled_copy, merged_allPs, on='p_id', how='left')

print(total_data_withcara.columns)


In [ ]:
# Specify the columns to include in the TableOne summary
columns = ['age', 'gender_all_x', 'IDS_TOTAL_x',# 'IDS_CAT',
           'recruitmentsite', 'mh_family_TOTAL', 'Mental_comorbidity',
       'Physical_comorbidity','WSAS_TOTAL', 'GAD7_TOTAL', 'EMPLOYMENT_STATUS'] 

# Create the TableOne grouped by HMM
table_activity_states = TableOne(total_data_withcara,                                       
                                      columns=columns, 
                                      categorical=['gender_all_x', 'recruitmentsite', 'mh_family_TOTAL',
                                                  'Mental_comorbidity',
                                                   'Physical_comorbidity', 'EMPLOYMENT_STATUS'],
                                      groupby='hmm',  # Group by 'hmm_mode'
                                     # pval=True
                                )
                                      #normal_test=True)  

# Display the grouped TableOne
print(table_activity_states)